In [34]:
import pandas as pd
import numpy as np
df=pd.read_csv('IMDB-Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [35]:
import re
import string
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()

    
    text=re.sub(r"<.*?>", " ", text)

    
    text=re.sub(r"http\S+|www\S+", " ", text)

    
    text=re.sub(r"\S+@\S+", " ", text)

    
    text=re.sub(f"[{re.escape(string.punctuation)}]", " ", text)

    
    text=re.sub(r"\d+", " ", text)

    
    text=re.sub(r"\s+", " ", text).strip()

    
    words=[word for word in text.split() if word not in stop_words]
    text= " ".join(words)

    return text

df['review']=df['review'].apply(clean_text)
df.head()


,review,sentiment
0,one reviewers mentioned watching oz episode ho...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive


In [51]:
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
tokenizer=tf.keras.layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=200
    
)
tokenizer.adapt(df['review'])
output=tokenizer(df['review'])

output=pd.DataFrame(output)
output

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,4,1901,917,56,2929,276,3035,99,489,467,...,0,0,0,0,0,0,0,0,0,0
1,271,42,235,1231,2811,1,62,7,2012,1429,...,0,0,0,0,0,0,0,0,0,0
2,90,271,23,1005,7,726,1317,2431,1087,755,...,0,0,0,0,0,0,0,0,0,0
3,552,120,42,279,3181,1099,850,4055,628,851,...,0,0,0,0,0,0,0,0,0,0
4,1,1,34,7,177,1979,1271,3,28,315,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,90,2,99,6,181,1368,106,20,863,115,...,0,0,0,0,0,0,0,0,0,0
49996,15,35,15,289,15,38,2868,824,500,1323,...,0,0,0,0,0,0,0,0,0,0
49997,3254,4206,1,8016,5304,9156,4206,1,9254,184,...,0,0,0,0,0,0,0,0,0,0
49998,70,2851,784,777,357,9934,4,219,818,1,...,0,0,0,0,0,0,0,0,0,0


In [52]:
voc_size=len(tokenizer.get_vocabulary())
input_len=len(output[0])
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , SimpleRNN , Dropout
model=Sequential()
model.add(tf.keras.layers.Embedding(input_dim=voc_size,output_dim=16,input_length=input_len,mask_zero=True))
model.add(SimpleRNN(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



c:\Users\amiya\.conda\envs\DL-new\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['sentiment']=le.fit_transform(df['sentiment'])
df.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [ ]:
x=output
y=df['sentiment']
from sklearn.model_selection import train_test_split
x_train , x_test ,y_train , y_test= train_test_split(x, y, test_size=0.25, random_state=42,shuffle=True)

In [53]:
import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
# log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%M%d-%H%M%S")
# TensorBoard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
EarlyStopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
history=model.fit(
   x_train,y_train,
   epochs =50,
   validation_split=0.25,
   callbacks=[EarlyStopping_callback]
)

Epoch 1/50


879/879 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - accuracy: 0.6087 - loss: 0.6468 - val_accuracy: 0.8075 - val_loss: 0.4487
Epoch 2/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.7913 - loss: 0.4589 - val_accuracy: 0.8457 - val_loss: 0.3802
Epoch 3/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.8989 - loss: 0.2659 - val_accuracy: 0.8530 - val_loss: 0.3777
Epoch 4/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9462 - loss: 0.1593 - val_accuracy: 0.7893 - val_loss: 0.5087
Epoch 5/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.9533 - loss: 0.1315 - val_accuracy: 0.6146 - val_loss: 0.8577
Epoch 6/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9620 - loss: 0.1075 - val_accuracy: 0.8308 - val_loss: 0.5979
Epoch 7/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.9876 - loss: 0.0423 - val_accuracy: 0.8162 - val_loss: 0.6962
Epoch 8/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9924 - loss: 0.0282 - val_accurac

In [ ]:
y_predict=model.predict(x_test)
y_predict


391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


array([[0.02185491],
       [0.98374665],
       [0.07988029],
       ...,
       [0.17037076],
       [0.9956971 ],
       [0.9651584 ]], dtype=float32)

In [61]:
for i in range(len(y_predict)):
      if y_predict[i]>=0.5:
         y_predict[i]=1
      else:
         y_predict[i]=0

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [66]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
accuracy_score(y_test,y_predict)


0.85624

In [67]:
model.save("review.keras")

In [69]:
vocab=tokenizer.get_vocabulary()
import pickle
pickle.dump(vocab,open("vocab.pkl",'wb'))